# Retrieve Dataset

In [19]:
%pip install PyDrive2
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

# Authenticate
gauth = GoogleAuth()
gauth.CommandLineAuth() # Opens a browser for OAuth login
drive = GoogleDrive(gauth)  # Create a GoogleDrive instance

Note: you may need to restart the kernel to use updated packages.


InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

Enter verification code:  4/1AVGzR1DtPHtkDMVlZUj0hEDDTPtpGrIqNmO6PVoiBHaTRjP9s1krtmNgAAw


Authentication successful.


In [3]:
# Replace with your file's ID
files = [
    ('17EIpH_0cVAETrDNcWciQaasBiHnvYKWm', 'dataset.zip'),
    ('19gIe0kIuHOBtRlcW2iEWc13MsiRAu1Xt', 'mmcv-2.1.0-cp312-cp312-linux_x86_64.whl')]

for (file_id, file_name) in files:
    # Create a file instance
    file = drive.CreateFile({'id': file_id})
    
    # Download the file
    file.GetContentFile(file_name)
    
    print(f"File downloaded as {file_name}")

File downloaded as dataset.zip
File downloaded as mmcv-2.1.0-cp312-cp312-linux_x86_64.whl


# Download dependencies

In [ ]:
!pip uninstall mmcv

In [7]:
# Download dependencies

!pip install ./mmcv-2.1.0-cp312-cp312-linux_x86_64.whl
!pip install mmsegmentation
!pip install mmengine
!pip install ftfy
!pip install mmpretrain
!pip install -U transformers==4.37.2
!pip install torch


Processing ./mmcv-2.1.0-cp312-cp312-linux_x86_64.whl
mmcv is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-ma

# Log into wandb

In [8]:
%pip install wandb
import wandb
wandb.login()

Note: you may need to restart the kernel to use updated packages.


True

# Unpack the data

In [5]:
!unzip ./dataset.zip

Archive:  ./dataset.zip
   creating: dataset/
   creating: dataset/holdout/
   creating: dataset/holdout/images_test/
  inflating: dataset/holdout/images_test/guatemala-volcano_00000004_pre_disaster.png  
  inflating: dataset/holdout/images_test/guatemala-volcano_00000012_pre_disaster.png  
  inflating: dataset/holdout/images_test/guatemala-volcano_00000014_pre_disaster.png  
  inflating: dataset/holdout/images_test/guatemala-volcano_00000020_pre_disaster.png  
  inflating: dataset/holdout/images_test/guatemala-volcano_00000022_pre_disaster.png  
  inflating: dataset/holdout/images_test/hurricane-florence_00000006_pre_disaster.png  
  inflating: dataset/holdout/images_test/hurricane-florence_00000009_pre_disaster.png  
  inflating: dataset/holdout/images_test/hurricane-florence_00000010_pre_disaster.png  
  inflating: dataset/holdout/images_test/hurricane-florence_00000011_pre_disaster.png  
  inflating: dataset/holdout/images_test/hurricane-florence_00000014_pre_disaster.png  
  infla

# TRAIN!!

In [18]:
#%env PYTHONPATH=/workspace/mmsegmentation_xview2
!python3 /workspace/mmsegmentation_xview2/tools/train.py /workspace/mmsegmentation_xview2/configs/convnext/convnext-tiny_upernet_xview2.py --work-dir /workspace/mmsegmentation_xview2/work_dirs/xview2_test


KeyboardInterrupt



In [13]:
import os
print(os.getcwd())

/workspace/mmsegmentation_xview2


In [ ]:

import glob, numpy as np, imageio
paths = glob.glob('dataset/masks/*.png') + glob.glob('dataset/masks_val/*.png') + glob.glob('dataset/holdout/*.png')
paths = [p for p in paths if p]
if not paths:
    print('No masks found for the given globs.')
    raise SystemExit(1)
cnt = {}
per_image_pos = []
for p in paths:
    m = imageio.imread(p)
    if m.ndim==3: m = m[...,0]
    vals, c = np.unique(m, return_counts=True)
    for v,cc in zip(vals,c): cnt[int(v)] = cnt.get(int(v),0) + int(cc)
    pos = int((m==1).sum()) if 1 in vals else 0
    per_image_pos.append(pos / m.size)
total = sum(cnt.values())
print('Global label counts:', {k: (v, v/total) for k,v in sorted(cnt.items())})
import numpy as np
arr = np.array(per_image_pos)
print(f'Per-image positive fraction mean={arr.mean():.6f}, std={arr.std():.6f}')
if 1 in cnt:
    print('Heuristic: value==1 fraction =', cnt[1]/total)
else:
    print('No pixels with value 1 found (inspect masks manually).')